In [ ]:
import requests
import numpy as np
import itertools

In [ ]:
class Config:
    class Room:
        ID = (f'{i}' for i in itertools.count())
        Types = ['Living Room', 'Kitchen', 'Bedroom', 'Bathroom', 'Balcony', 'Entrance', 'Dining Room', 'Study Room', 'Storage']

    class Edge:
        ID = (f'{i}' for i in  itertools.count())

In [ ]:
def Node(room_type = None, corners = None):
    if room_type is None or room_type not in Config.Room.Types: room_type = np.random.choice(Config.Room.Types)
    if corners is None: corners = int(np.random.choice([4,6], p=[1.,0.]))
    return {
      "id": next(Config.Room.ID), # Nodes unique id   
      "room_type": room_type, # Room Type in ['Living Room', 'Kitchen', 'Bedroom', 'Bathroom', 'Balcony', 'Entrance', 'Dining Room', 'Study Room', 'Storage', 'Front Door', 'Unknown', 'Interior Door']
      "corners": corners # Number of Corners
    }

In [ ]:
rooms = [Node(*conf) for conf in [(),(),(),(),()]]

edges = []

import igraph as ig

g = ig.Graph.Full(len(rooms))
weights = list(range(g.ecount()))
np.random.shuffle(weights)
g.es["weight"] = weights

for edge in g.spanning_tree(weights=g.es["weight"], return_tree=False):
  a,b = g.es[edge].source, g.es[edge].target
  if a<b:
    edges.append({
      "id": next(Config.Edge.ID), # Edge Unique id
      "source": rooms[a]['id'], # First Nodes id
      "target": rooms[b]['id']  # Second Nodes id
    })

In [ ]:
payload = {
  "nodes": rooms,
  "edges": edges,
  "metrics": False # If True provides the length and width of rooms in pixels
}

In [ ]:
payload

In [ ]:
req = requests.post('http://0.0.0.0:8000/generate', json=payload)

In [ ]:
res = req.json()

In [ ]:
from IPython import display
from base64 import b64decode

svg = res['dataUri'][0]
display.HTML(f'<img src="{svg}" />')

In [ ]:
import io
import PIL.Image
import cairosvg

img = io.BytesIO()
cairosvg.svg2png(url=svg, write_to=img, scale=5)
gray = np.array(PIL.Image.open(img)).mean(axis=-1).astype(int)

occupied = np.argwhere(gray != 255)
top_left = occupied.min(axis=0)
bottom_right = occupied.max(axis=0)
gray = gray[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]

In [ ]:
from matplotlib import pyplot as plt


plt.imshow(gray>0, cmap='gray')